## Example of the `aitlas` toolbox in the context of image segmentation


### Import the required packages

In [ ]:
from aitlas.datasets import TiiLIDARDataset
from aitlas.models import HRNet


### Load train, validation and test data

In [ ]:
train_dataset_config = {
    "batch_size": 16,
    "shuffle": True,
    "num_workers": 4,
    "data_dir": "/home/dragik/IrishArchaeology/samples_256px/train/segmentation/seg_SLRM/output",
    "csv_file": "/home/dragik/IrishArchaeology/samples_256px/train/segmentation/seg_SLRM/train.txt",
    "transforms": ["aitlas.transforms.MinMaxNormTranspose"],
    "joint_transforms": ["aitlas.transforms.FlipHVRandomRotate"],
    "target_transforms": ["aitlas.transforms.Transpose"]
}
train_dataset = TiiLIDARDataset(train_dataset_config)

validation_dataset_config = {
    "batch_size": 16,
    "shuffle": False,
    "num_workers": 4,
    "data_dir": "/home/dragik/IrishArchaeology/samples_256px/validation/segmentation/seg_SLRM/output",
    "csv_file": "/home/dragik/IrishArchaeology/samples_256px/validation/segmentation/seg_SLRM/validation.txt",
    "transforms": ["aitlas.transforms.MinMaxNormTranspose"],
    "target_transforms": ["aitlas.transforms.Transpose"]
}
validation_dataset = TiiLIDARDataset(validation_dataset_config)

test_dataset_config = {
    "batch_size": 16,
    "shuffle": False,
    "num_workers": 4,
    "data_dir": "/home/dragik/IrishArchaeology/samples_256px/test/segmentation/seg_SLRM/output",
    "csv_file": "/home/dragik/IrishArchaeology/samples_256px/test/segmentation/seg_SLRM/test.txt",
    "transforms": ["aitlas.transforms.MinMaxNormTranspose"],
    "target_transforms": ["aitlas.transforms.Transpose"]
}
test_dataset = TiiLIDARDataset(test_dataset_config)

len(train_dataset), len(validation_dataset), len(test_dataset)

### Create the model

In [ ]:
epochs = 50
model_directory = "/home/dragik/aitlas_v1/experiments/semantic_segmentation/"
model_config = {
    "num_classes": 3,
    "learning_rate": 0.0001,
    "pretrained": True,
    "threshold": 0.5,
    "metrics": ["iou"]
}

model = HRNet(model_config)
model.prepare()

### Train the model

In [ ]:
model.train_and_evaluate_model(
    train_dataset=train_dataset,
    val_dataset=validation_dataset,
    epochs=epochs,
    model_directory=model_directory,
    save_epochs=10,
    run_id='1'
);

### Evaluate the model on test data

In [ ]:
model = HRNet(model_config)
model.prepare()
model.running_metrics.reset()
model_path = "/home/dragik/aitlas_v1/experiments/semantic_segmentation/train_HRNet_SLRM_256px_pretrained/best_checkpoint_1685061651_30.pth.tar"
model.evaluate(dataset=test_dataset, model_path=model_path)
model.running_metrics.get_scores(model.metrics)